# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
import findspark
import os
spark_location='/home/rakaaadli/spark-2.4.5-bin-hadoop2.7' # Set your own I'm using ubuntu
java8_location= '/usr/lib/jvm/java-8-openjdk-amd64' # Set your own I'm using ubuntu
os.environ['JAVA_HOME'] = java8_location
findspark.init(spark_home=spark_location)

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [7]:
data.describe()

DataFrame[summary: string, movieId: string, rating: string, userId: string]

We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [8]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [9]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-------------+
|movieId|rating|userId|   prediction|
+-------+------+------+-------------+
|     31|   1.0|    18|    1.7322211|
|     85|   2.0|    20|     2.218995|
|     65|   1.0|    22|   0.92719835|
|     65|   1.0|    19|     2.365151|
|     78|   1.0|     1|   0.84254766|
|     78|   1.0|    13|   0.96781236|
|     78|   1.0|    20|   0.91533077|
|     78|   1.0|    19|   0.77946913|
|     34|   1.0|    28|-0.0012560934|
|     34|   3.0|     3|    1.9271514|
|     34|   1.0|    19|    1.6707436|
|     34|   4.0|     2|    1.3911974|
|     81|   1.0|    19|    1.0741336|
|     81|   1.0|    21|    1.2143126|
|     28|   1.0|     6|     2.372019|
|     76|   3.0|     3|    0.5502058|
|     76|   3.0|     7|    2.7457087|
|     26|   1.0|    13|     1.005351|
|     27|   1.0|    21|    0.9521968|
|     44|   3.0|    27|    1.8108695|
+-------+------+------+-------------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.5069472234664678


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [13]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [14]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|      9|    11|
|     16|    11|
|     20|    11|
|     22|    11|
|     32|    11|
|     64|    11|
|     67|    11|
|     69|    11|
|     90|    11|
|     99|    11|
+-------+------+



In [15]:
reccomendations = model.transform(single_user)

In [16]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     32|    11|    2.071854|
|     16|    11|   1.2637478|
|     99|    11|   0.8963011|
|     22|    11|   0.8731635|
|     64|    11|   0.5523911|
|      9|    11|  0.50718534|
|      6|    11|   0.0506019|
|     67|    11| 0.025168657|
|     69|    11|-0.054492235|
|     20|    11| -0.29727596|
|     90|    11|  -1.0278728|
+-------+------+------------+



# Great Job!